In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
# Fork-Join queueing network
model = Network('model')

In [3]:
# Create network 
source = Source(model, 'Source')

queue1 = Queue(model, 'Queue1', SchedStrategy.PS)
queue2 = Queue(model, 'Queue2', SchedStrategy.PS)

fork = Fork(model, 'Fork')
fork.set_tasks_per_link(2)
join = Join(model, 'Join', fork)

sink = Sink(model, 'Sink')

In [4]:
# Create job classes
jobclass1 = OpenClass(model, 'class1')
jobclass2 = OpenClass(model, 'class2')

In [5]:
#Set arrival and service processes
# Class 1
source.set_arrival(jobclass1, Exp(0.25))
queue1.set_service(jobclass1, Exp(1.0))
queue2.set_service(jobclass1, Exp(0.75))
# Class 2
source.set_arrival(jobclass2, Exp(0.25))
queue1.set_service(jobclass2, Immediate())
queue2.set_service(jobclass2, Exp(2.0))

In [6]:
# Set routing matrix
P = model.init_routing_matrix() # Class 1 routing
P.set(jobclass1, jobclass1, source, fork, 1.0)
P.set(jobclass1, jobclass1, fork, queue1, 1.0)
P.set(jobclass1, jobclass1, fork, queue2, 1.0)
P.set(jobclass1, jobclass1, queue1, join, 1.0)
P.set(jobclass1, jobclass1, queue2, join, 1.0)
P.set(jobclass1, jobclass1, join, sink, 1.0) # Class 2 routing
P.set(jobclass2, jobclass2, source, fork, 1.0)
P.set(jobclass2, jobclass2, fork, queue1, 1.0)
P.set(jobclass2, jobclass2, fork, queue2, 1.0)
P.set(jobclass2, jobclass2, queue1, join, 1.0)
P.set(jobclass2, jobclass2, queue2, join, 1.0)
P.set(jobclass2, jobclass2, join, sink, 1.0)
model.link(P)

In [7]:
# Solve with multiple methods
solvers = [ JMT(model, seed=23000), MVA(model)]
for i, solver in enumerate(solvers):
    print(f'SOLVER {i+1}: {solver.get_name()}')
    avgTable = solver.avg_table()

SOLVER 1: JMT


Station JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
 Source   class1 0.0000 0.0000 0.0000  0.0000 0.0000 0.2500
 Source   class2 0.0000 0.0000 0.0000  0.0000 0.0000 0.2500
 Queue1   class1 0.3273 0.2336 1.2943  1.2943 0.2491 0.2491
 Queue1   class2 0.0000 0.0000 0.0000  0.0000 0.2510 0.2510
 Queue2   class1 0.6049 0.3458 2.5167  2.5167 0.2472 0.2472
 Queue2   class2 0.2273 0.1204 0.8964  0.8964 0.2510 0.2510
   Join   class1 0.5438 0.3518 1.0632  1.0632 0.2472 0.2472
   Join   class2 0.2273 0.1765 0.4227  0.4227 0.2510 0.2510
SOLVER 2: MVA
Station JobClass   QLen   Util  RespT  ResidT  ArvR  Tput
 Source   class1 0.0000 0.0000 0.0000  0.0000  0.00  0.25
 Source   class2 0.0000 0.0000 0.0000  0.0000  0.00  0.25
 Queue1   class1 0.3333 0.2500 1.3333  1.3333  0.25  0.25
 Queue1   class2 0.0000 0.0000 0.0000  0.0000  0.25  0.25
 Queue2   class1 0.5000 0.3333 2.0000  2.0000  0.25  0.25
 Queue2   class2 0.1429 0.1250 0.5714  0.5714  0.25  0.25
   Join   class1 0.4333 0.0000 0.8667  0